In [1]:
import torch
from model import *
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

# Load Model

In [2]:
model = SymbolDetector()
model_path = 'resized_model_50.pth'
# model_path = 'resized_model_10.pth'
# model_path = 'quantized_model.pth'

In [3]:
model.load_state_dict(torch.load(model_path))
model = model.cuda()
print("Model loaded Successfully!")

Model loaded Successfully!


In [4]:
def general_test(model, resize=False):
    model.eval()
    # Traverse the 'test_data' directory
    test_dir = 'test_data'
    if resize:
        test_dir = 'resized_test_data'
    for root, dirs, files in os.walk(test_dir):
        for file in files:
            image_path = os.path.join(root, file)
            image_size = 256
            image = Image.open(image_path)
            # image = image.convert('RGB')
            original_size = image.size
            transform = transforms.Compose([
                    transforms.Resize((image_size, image_size)),
                    transforms.ToTensor(),
                    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            image = transform(image).unsqueeze(0).cuda()
            with torch.no_grad():
                presence_pred, bbox_pred = model(image)
                print(f'Presence_pred{presence_pred}, Bbox_pred{bbox_pred}')


            threshold = 0.5
            symbol_present = (presence_pred > threshold).float()
            print(f'Symbol Present: {symbol_present}')

            image_cv = cv2.imread(image_path)
            # image_cv = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for visualization
            image_cv = np.array(image_cv)

            if symbol_present.item() == 1:
                prediced_boxes = bbox_pred[0].cpu().numpy()
                # Extract bounding box coordinates
                x_min, y_min, x_max, y_max = prediced_boxes
                if resize:
                    ...
                else:
                    x_min = x_min * original_size[0] / image_size
                    y_min = y_min * original_size[1] / image_size
                    x_max = x_max * original_size[0] / image_size
                    y_max = y_max * original_size[1] / image_size
                print(f"Bounding Box Coordinates: {x_min, y_min, x_max, y_max}")

                # Draw the bounding box on the image
                cv2.rectangle(image_cv, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 0), 2)
                cv2.imshow('Augmented Image', image_cv)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            else:
                center = (original_size[0] // 2, original_size[1] // 2)
                cv2.putText(image_cv, "No Symbol Detected", center, cv2.FONT_HERSHEY_SIMPLEX, 1, (125, 0, 0), 2)
                cv2.imshow('Augmented Image', image_cv)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


In [5]:
# general_test(model)
general_test(model)

Presence_predtensor([[0.9999]], device='cuda:0'), Bbox_predtensor([[ 12.9686,  26.0681,  64.3862, 110.6626]], device='cuda:0')
Symbol Present: tensor([[1.]], device='cuda:0')
Bounding Box Coordinates: (52.482107400894165, 65.78131729364395, 260.56302642822266, 279.2501680254936)
Presence_predtensor([[1.0000]], device='cuda:0'), Bbox_predtensor([[249.2210, 227.7156, 199.2603, 154.2418]], device='cuda:0')
Symbol Present: tensor([[1.]], device='cuda:0')
Bounding Box Coordinates: (1008.5661444664001, 574.6260242462158, 806.3816487789154, 389.2195574045181)
Presence_predtensor([[3.7636e-12]], device='cuda:0'), Bbox_predtensor([[189.8237, 128.8186, 163.3060, 127.4560]], device='cuda:0')
Symbol Present: tensor([[0.]], device='cuda:0')
Presence_predtensor([[3.3830e-12]], device='cuda:0'), Bbox_predtensor([[191.0873, 132.3286, 163.9569, 128.1853]], device='cuda:0')
Symbol Present: tensor([[0.]], device='cuda:0')
Presence_predtensor([[9.4987e-12]], device='cuda:0'), Bbox_predtensor([[188.7850, 1

In [ ]:
# model.eval()
# image_path = 'front.bmp'
# image_size = 256
# image = Image.open(image_path).convert('RGB')
# original_size = image.size
# transform = transforms.Compose([
#         transforms.Resize((image_size, image_size)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
# image = transform(image).unsqueeze(0).cuda()
# with torch.no_grad():
#     presence_pred, bbox_pred = model(image)
#     print(f'Presence_pred{presence_pred}, Bbox_pred{bbox_pred}')


In [ ]:
# threshold = 0.5
# symbol_present = (presence_pred > threshold).float()
# print(f'Symbol Present: {symbol_present}')

# image_cv = cv2.imread(image_path)
# # image_cv = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for visualization
# image_cv = np.array(image_cv)

# if symbol_present.item() == 1:
#     prediced_boxes = bbox_pred[0].cpu().numpy()
#     # Extract bounding box coordinates
#     x_min, y_min, x_max, y_max = prediced_boxes
#     x_min = x_min * original_size[0] / image_size
#     y_min = y_min * original_size[1] / image_size
#     x_max = x_max * original_size[0] / image_size
#     y_max = y_max * original_size[1] / image_size
#     print(f"Bounding Box Coordinates: {x_min, y_min, x_max, y_max}")

#     # Draw the bounding box on the image
#     cv2.rectangle(image_cv, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 0), 2)
#     cv2.imshow('Augmented Image', image_cv)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# else:
#     center = (original_size[0] // 2, original_size[1] // 2)
#     cv2.putText(image_cv, "No Symbol Detected", center, cv2.FONT_HERSHEY_SIMPLEX, 1, (125, 0, 0), 2)
#     cv2.imshow('Augmented Image', image_cv)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# Onnx

In [6]:
import torch.onnx
import onnx, onnxruntime

In [7]:
model.eval()
batch_size = 1
image_size = 256
dummy_input = torch.randn(batch_size, 3, image_size, image_size)
dummy_input = dummy_input.to('cuda')
torch.onnx.export(model,
                  dummy_input,
                  "onnx_model.onnx",
                  export_params=True,
                  opset_version=10, # the ONNX version to export the model to
                  do_constant_folding=True, # whether to execute constant folding for optimization
                  input_names=['image'],
                  output_names=['presence_pred', 'bbox_pred'], 
                  dynamic_axes={'image' : {0 : 'batch_size'},    # variable lenght axes
                                'presence_pred' : {0 : 'batch_size'},
                                'bbox_pred' : {0 : 'batch_size'}})

1) Check Onnx Model

In [8]:
onnx_model = onnx.load("onnx_model.onnx")
onnx.checker.check_model(onnx_model)
torch_presence, torch_bbox = model(dummy_input)

2) compare with Pytorch Model

In [9]:
ort_session = onnxruntime.InferenceSession("onnx_model.onnx", provider_options={'device_type': 'cuda'})
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_presence, ort_bbox = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_presence), ort_presence, rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(to_numpy(torch_bbox), ort_bbox, rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")


Exported model has been tested with ONNXRuntime, and the result looks good!


3) test onnx model using image

In [10]:
image_path = 'back.bmp'
image = Image.open(image_path) # RGB

# print(image.size)
# image_array = np.array(image)
# print(image_array.shape)
# for i in range(10):
#     for j in range(10):
#         print(f"Index {i, j}: {image_array[i, j]}")

original_size = image.size
image_size = 256
transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = transform(image) # C x W x H
# image = image.permute(1, 2, 0)  # Convert image from C x W x H to W x H x C
# image = image.numpy()  # Convert image to numpy array
# image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert RGB image to BGR
# image = torch.from_numpy(image)  # Convert image back to torch tensor
# image = image.permute(2, 0, 1)  # Convert image from W x H x C to C x W x H
# print(image.shape)
# for i in range(10):
#     for j in range(10):
#         for k in range(3):
#             print(f"Index {k, i, j}: {image[k, i, j]}")
    


image = image.unsqueeze(0).cuda() # 1 x C x W x H



ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}
ort_outs = ort_session.run(None, ort_inputs)

print(ort_outs)

image_cv = cv2.imread(image_path)
# image_cv = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for visualization
image_cv = np.array(image_cv)

# Extract bounding box coordinates
presence = ort_outs[0][0]
x_min, y_min, x_max, y_max = ort_outs[1][0]

x_min = x_min * original_size[0] / image_size
y_min = y_min * original_size[1] / image_size
x_max = x_max * original_size[0] / image_size
y_max = y_max * original_size[1] / image_size

print(f"Bounding Box Coordinates: {x_min, y_min, x_max, y_max}")
# Draw the bounding box on the image
if presence > 0.5:
        cv2.rectangle(image_cv, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 0), 2)
        cv2.imshow('Augmented Image', image_cv)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

[array([[0.9999834]], dtype=float32), array([[ 12.866848,  25.555843,  64.27002 , 110.729385]], dtype=float32)]
Bounding Box Coordinates: (52.07052546739578, 64.48857346177101, 260.09273529052734, 279.41868340969086)


# Pytorch test


In [ ]:
# model.eval()
# image_size = 256

# transform = transforms.Compose([
#         # transforms.Resize((image_size, image_size)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
# # Test the model
# image = Image.open(image_path).convert('RGB')
# image = transform(image)

# image = image.unsqueeze(0)


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
# image = image.to(device)

# with torch.no_grad():
#     output = model(image)

# prediced_boxes = output[0].cpu().numpy()
# print(f"Predicted Boxes: {prediced_boxes}")

In [ ]:
# # Convert the image to OpenCV format for visualization
# image_cv = cv2.imread(image_path)
# # image_cv = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for visualization
# image_cv = np.array(image_cv)

# # Extract bounding box coordinates
# x_min, y_min, x_max, y_max = prediced_boxes
# x_min = x_min * original_size[0] / image_size
# y_min = y_min * original_size[1] / image_size
# x_max = x_max * original_size[0] / image_size
# y_max = y_max * original_size[1] / image_size
# print(f"Bounding Box Coordinates: {x_min, y_min, x_max, y_max}")

# # Draw the bounding box on the image
# cv2.rectangle(image_cv, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 0), 2)
# cv2.imshow('Augmented Image', image_cv)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
